# Título   Crear clase ternaria con duckdb
## Autor: Jose Chelquer
## Fecha de última modificación: 22/10/2025
## Descripción:

Crear clase ternaria usando duckdb

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [4]:
#usar_gdrive=False    # Poner en true si se va a ejecutar con archivos de google drive
usar_gdrive=True

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [5]:
dataset_path='/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/'
#dataset_path = '~/buckets/b1/datasets'
dataset_file='competencia_01.csv'
#dataset_file = 'competencia_02_crudo.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [6]:
output_file='competencia_02_duck.csv.gz'
duckdb_file='competencia_02.db'

## Procesos

### Paquetes necesarios

In [7]:
!pip install duckdb

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [8]:
import os
import pandas as pd
import duckdb

In [9]:
if usar_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
file_path = os.path.join(dataset_path, dataset_file)
# Conectar a una base de datos en memoria o en un archivo
duckdb_path=os.path.join(dataset_path, duckdb_file)

conn = duckdb.connect(duckdb_path)


In [14]:
# Dado un mes en formato aaaamm y una cantidad de meses
# devuelve el mes a distancia meses de mes
def sig_mes(mes: int, meses: int) -> int:
  aaaa=mes//100
  mm=mes%100+meses
  aExtra=(mm-1)//12
  mm=mm-12*aExtra
  return (aaaa+aExtra)*100+mm



In [15]:

# Registrar la función UDF en DuckDB
conn.create_function('sig_mes', sig_mes)

CatalogException: Catalog Error: Scalar Function with name "sig_mes" already exists!

## Leer datos en tabla

In [18]:
def do_sql(sql):
  print (sql)
  resultado=conn.execute(sql).fetchall()
  print (resultado)

In [16]:
# Leer el CSV y cargarlo en una tabla de DuckDB
sql=f"DROP TABLE IF EXISTS competencia_02"
conn.execute(sql)
sql =f"CREATE TABLE competencia_02 AS SELECT * FROM read_csv_auto('{file_path}')"
print (sql)
conn.execute(sql)



CREATE TABLE competencia_02 AS SELECT * FROM read_csv_auto('/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/competencia_01.csv')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
# Verificar si la tabla fue creada correctamente
result = conn.execute("SELECT * FROM competencia_02 LIMIT 5").fetchall()
print(result)
# Contar los registros en la tabla
count = conn.execute("SELECT COUNT(*) FROM competencia_02").fetchone()[0]
print (f'Count:{count}')



[(260486881, 202101, 1, 0, 0, 57, 85, 2014.41, 23213.4, 2542.52, -1128.1, 677.88, 6, 1, 1, 0.0, 0.0, 2, 20635.83, 0.0, 0.0, 1, 26175.28, 1, 0, 0.0, 1, 17, 22930.67, 0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0.0, 0, 0.0, 0.0, 0, 0.0, 0, 0, 1, 0, 0, 0, 0.0, 0.0, 0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 1, 1364.94, 6, 2542.52, 0, 0, 0.0, 0, 0.0, 1, 16422.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0, 1, 25, 0, 0, 0, 0, 0, 0, 0.0, 0, 0.0, 44, 0, 0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 0.0, 0.0, 152020.8, -1825.0, None, 19873.35, 23311.43, 0.0, 6221.24, 1.95, 168912.0, 0.0, 0.0, 4.0, 0.0, -15207.76, 0.0, 2573.0, 6221.24, 8.0, 0.0, 1982.37, 'CONTINUA'), (260486881, 202102, 1, 0, 0, 57, 86, 1505.51, 22577.48, 2180.71, -826.27, 250.65, 6, 1, 1, 0.0, 0.0, 2, 9906.41, 0.0, 0.0, 1, 21061.16, 1, 0, 0.0, 1, 17, 25157.0, 0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0.0, 0

## Calcular clase ternaria

In [21]:
# Crear campos auxiliares
sql="""
ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS existe1 BOOLEAN;
"""
do_sql(sql)
sql="""
ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS existe2 BOOLEAN;
"""
do_sql(sql)
sql="""
ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS ternaria_propuesta STRING;
"""
do_sql(sql)




ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS existe1 BOOLEAN;

[]

ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS existe2 BOOLEAN;

[]

ALTER TABLE competencia_02
ADD COLUMN IF NOT EXISTS ternaria_propuesta STRING;

[]


In [22]:
# existe1
sql="""
UPDATE competencia_02 c1
SET existe1 = EXISTS (
    SELECT 1
    FROM competencia_02 c2
    WHERE c2.numero_de_cliente = c1.numero_de_cliente
      AND c2.foto_mes = sig_mes(c1.foto_mes, 1)
);
"""
do_sql(sql)


UPDATE competencia_02 c1
SET existe1 = EXISTS (
    SELECT 1
    FROM competencia_02 c2
    WHERE c2.numero_de_cliente = c1.numero_de_cliente
      AND c2.foto_mes = sig_mes(c1.foto_mes, 1)
);

[(981946,)]


In [23]:
# existe 2
sql="""
UPDATE competencia_02 c1
SET existe2 = EXISTS (
    SELECT 1
    FROM competencia_02 c2
    WHERE c2.numero_de_cliente = c1.numero_de_cliente
      AND c2.foto_mes = sig_mes(c1.foto_mes, 2)
);
"""
do_sql(sql)


UPDATE competencia_02 c1
SET existe2 = EXISTS (
    SELECT 1
    FROM competencia_02 c2
    WHERE c2.numero_de_cliente = c1.numero_de_cliente
      AND c2.foto_mes = sig_mes(c1.foto_mes, 2)
);



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[(981946,)]


In [25]:
# clase ternaria
sql="""
UPDATE competencia_02
SET ternaria_propuesta = CASE
    WHEN existe2 = TRUE THEN 'CONTINUA'
    WHEN existe1 = FALSE THEN 'BAJA+1'
    WHEN existe1 = TRUE AND existe2 = FALSE THEN 'BAJA+2'
END;
"""
do_sql(sql)


UPDATE competencia_02
SET ternaria_propuesta = CASE
    WHEN existe2 = TRUE THEN 'CONTINUA'
    WHEN existe1 = FALSE THEN 'BAJA+1'
    WHEN existe1 = TRUE AND existe2 = FALSE THEN 'BAJA+2'
END;

[(981946,)]


In [30]:
# elimina auxiliares
sql="""
ALTER TABLE competencia_02
DROP COLUMN IF EXISTS existe1;
"""
do_sql(sql)
sql="""
ALTER TABLE competencia_02
DROP COLUMN IF EXISTS existe2;
"""
do_sql(sql)


ALTER TABLE competencia_02
DROP COLUMN IF EXISTS existe1;

[]

ALTER TABLE competencia_02
DROP COLUMN IF EXISTS existe2;

[]


## Graba output

In [ ]:
sql="""
COPY competencia_02 TO (SELECT dataset_path || '/' || output_file || '.csv')
    WITH (FORMAT CSV, HEADER TRUE, COMPRESSION GZIP);
"""
do_sql(sql)

In [ ]:
# Cerrar la conexión cuando termines
conn.close()